# Clustering users

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

import sklearn
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import BisectingKMeans
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading

from sklearn.metrics import davies_bouldin_score
from sklearn import datasets
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import json
import ast

import datetime
from dateutil import parser

import re

import bz2

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

#from profanity_check import predict, predict_prob
#from better_profanity import profanity

#from feature_creation import create

## Analyze the features from users

In [ ]:
df_id = pd.read_csv("new_outputs/all_user_features.csv")
#df = pd.read_csv("new_outputs/user_features_20220601.csv")
#df = df.append(pd.read_csv("new_outputs/user_features.csv"))

df_guesses = pd.read_csv("new_outputs/guesses.csv", dtype=str)

In [ ]:
# len(df[df["avg_num_lists_per_day"] == float('inf')])
# df["avg_num_lists_per_day"] = df["n_lists"] / df["days_since_creation"]
# df.to_csv("new_outputs/all_user_features.csv", index=False)

In [ ]:
df = df_id.drop(['stream_date', 'uid'], axis=1)

In [ ]:
df_id.loc[(df_id['mean_jaro_winkler_similarity'] >= 0.8) & (df_id['rt_ratio'] >= 0.8)]

In [ ]:
df

In [ ]:
df = df.loc[df['rt_ratio'] >= 0.8]
df_id = df_id.loc[df_id['rt_ratio'] >= 0.8]

In [ ]:
labels = []
ids = df_id['uid'].to_numpy(dtype=str)
gus = df_guesses['user_id'].to_numpy(dtype=str)

for elem in ids:
    if elem in gus:
        labels.append(0)
    elif "1468620673115373569" == elem or "1327935633445199874" == elem or "1511590589405536259" == elem:
        labels.append(1)
    else:
        labels.append(-1)
y = np.array(labels)

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
N_CLUSTERS = 2
#clustering = KMeans(n_clusters=N_CLUSTERS, random_state=5)
#clustering = BisectingKMeans(n_clusters=N_CLUSTERS, random_state=5)
clustering = AgglomerativeClustering(n_clusters=N_CLUSTERS)

# svc = SVC(probability=True, gamma="auto")
# clustering = SelfTrainingClassifier(svc)
# clustering = LabelPropagation()
# clustering = LabelSpreading(n_neighbors=2, gamma=0.1, alpha=0.0001,max_iter=100, tol=1e-8)

names = df.columns
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df, columns=names)
# Used to remove outliars
# scaled_df = scaled_df.loc[scaled_df['avg_rt_on_tweets'] <= 6]

X = scaled_df.to_numpy()
# Apply PCA such that 95% of the variance is retained

#pca = PCA(0.95)
#pca.fit(X)
#print(pca.n_components_)
#X = pca.transform(X)

clustering.fit(X, y)

model = clustering.fit(X, y)

# plt.figure(figsize=(16,9))
# plt.title("Hierarchical Clustering Dendrogram")
# # plot the top three levels of the dendrogram
# plot_dendrogram(model, truncate_mode="level", p=3)
# plt.xlabel("Number of points in node (or index of point if no parenthesis).")
# #plt.savefig('dendorgam.png', facecolor="white")
# plt.show()

In [ ]:
labels = model.fit_predict(X)

In [ ]:
df_idnp = df_id['uid'].to_numpy(dtype=str)
for l, id in zip(labels, df_idnp):
    print(l)
    if id in ["1468620673115373569", "1327935633445199874", "1511590589405536259"]:
        pass
        #print(l)

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = OPTICS(max_eps=10*n_clusters)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # # 2nd Plot showing the actual clusters formed
    # colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    # ax2.scatter(
    #     X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    # )

    # # Labeling the clusters
    # centers = clusterer.cluster_centers_
    # # Draw white circles at cluster centers
    # ax2.scatter(
    #     centers[:, 0],
    #     centers[:, 1],
    #     marker="o",
    #     c="white",
    #     alpha=1,
    #     s=200,
    #     edgecolor="k",
    # )

    # for i, c in enumerate(centers):
    #     ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    # ax2.set_title("The visualization of the clustered data.")
    # ax2.set_xlabel("Feature space for the 1st feature")
    # ax2.set_ylabel("Feature space for the 2nd feature")

    # plt.suptitle(
    #     "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
    #     % n_clusters,
    #     fontsize=14,
    #     fontweight="bold",
    # )

plt.show()

In [ ]:
len(labels[labels == 0])

In [ ]:
guesses = df_guesses["user_id"].to_numpy()

index_class_1 = np.where(model.labels_ == 2)
classified = df_id.iloc[index_class_1]["uid"].to_numpy(dtype=str)

count = 0
for i in guesses:
    count += i in classified

print(count, len(model.labels_[model.labels_ == 0]))

In [ ]:
np_labels = np.array(labels == 0)
np.array(labels == 0)

In [ ]:
pairs = {(x, y) for x in df.columns for y in df.columns if "context_domain" not in x and "context_domain" not in y and "lang" not in x and "lang" not in y}
len(pairs)

In [ ]:
Y = df.to_numpy()
count=0
ALPHA = 0.5
#for x_feature, y_feature in pairs:
x_feature = "mean_lcs"
y_feature = "avg_rt_on_tweets"

plt.figure(figsize=(16,9))

plt.xlabel(x_feature)
plt.ylabel(y_feature)

CLASS_1 = 0
CLASS_2 = 1

dataframe = scaled_df
labels = clustering.labels_

if N_CLUSTERS == 3:
    plt.scatter(dataframe[x_feature].to_numpy()[labels == 2], dataframe[y_feature].to_numpy()[labels == 2], c='g', alpha=ALPHA, label=f"{len(dataframe[x_feature].to_numpy()[labels == 2])} users")
plt.scatter(dataframe[x_feature].to_numpy()[labels == 0], dataframe[y_feature].to_numpy()[labels == 0], c='r', alpha=ALPHA, label=f"{len(dataframe[x_feature].to_numpy()[labels == 0])} users")
plt.scatter(dataframe[x_feature].to_numpy()[labels == 1], dataframe[y_feature].to_numpy()[labels == 1], c='b', alpha=ALPHA, label=f"{len(dataframe[x_feature].to_numpy()[labels == 1])} users")

# IF WE WANT TO USE FITERING
# CLASS_1 = 0
# CLASS_2 = 1

# filter = scaled_df[y_feature] < 12
# dataframe = scaled_df[filter]
# f_labels = labels[np.where(filter)]

# if N_CLUSTERS == 3:
#     plt.scatter(dataframe[x_feature].to_numpy()[f_labels == 2], dataframe[y_feature].to_numpy()[f_labels == 2], c='g', alpha=ALPHA, label=f"{len(dataframe[x_feature].to_numpy()[f_labels == 2])} users")
# plt.scatter(dataframe[x_feature].to_numpy()[f_labels == 0], dataframe[y_feature].to_numpy()[f_labels == 0], c='r', alpha=ALPHA, label=f"{len(dataframe[x_feature].to_numpy()[f_labels == 0])} users")
# plt.scatter(dataframe[x_feature].to_numpy()[f_labels == 1], dataframe[y_feature].to_numpy()[f_labels == 1], c='b', alpha=ALPHA, label=f"{len(dataframe[x_feature].to_numpy()[f_labels == 1])} users")

plt.legend()

plt.savefig(f'{x_feature}_{y_feature}_{N_CLUSTERS}.png', facecolor="white")
plt.savefig(f'figs/{x_feature}_{y_feature}_{N_CLUSTERS}.png', facecolor="white")


plt.show()
plt.clf()
plt.close()
#print(count)
#count+=1


In [ ]:
new_df = scaled_df.copy().assign(cls=labels)
new_df

In [ ]:
new_df = new_df.loc[:, new_df.any()]
new_df

# Interactive parallel coordinates

In [ ]:
import plotly.express as px
#fig = px.parallel_coordinates(new_df, dimensions=new_df.columns[150:-1], color="cls")
fig = px.parallel_coordinates(new_df, dimensions=new_df.columns[0:-1], color="cls")
fig.update_layout(width=8000)
fig.show(renderer="browser")

# Compute the distance of each data point to centroid

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
distances = euclidean_distances(scaled_df, clustering.cluster_centers_)

In [ ]:
max_distance = 20
filter_contition = ((distances[:,0] < max_distance) & (distances[:,1] < max_distance))
filtered_distances = distances[filter_contition]
filtered_distances.shape
# 8842-8305 = 537 going to max 20

In [ ]:
plt.figure(figsize=(20,20))
plt.title("Distance to each cluster")

plt.plot([0,max_distance], [0,max_distance], 'k-')
plt.scatter(filtered_distances[:, 0], filtered_distances[:, 1], c=labels[filter_contition])

plt.xlabel("Class 1")
plt.ylabel("Class 2")

#plt.savefig(f'cluster_distances_max_20.png', facecolor="white")

plt.show()

In [ ]:
sum_of_squared_distances = []
silhouette_coefficient = []
calinski_harabasz = []
davies_bouldin = []
K = range(2,30)
for k in K:
    km = BisectingKMeans(n_clusters=k, random_state=5)
    km = km.fit(X)
    silhouette_coefficient.append(metrics.silhouette_score(X, km.labels_, metric='euclidean'))
    calinski_harabasz.append(metrics.calinski_harabasz_score(X, km.labels_))
    davies_bouldin.append(davies_bouldin_score(X, km.labels_))
    sum_of_squared_distances.append(km.inertia_)

In [ ]:
len(df)

In [ ]:
plt.plot(K, davies_bouldin, 'bx-')
plt.xlabel('k')
plt.ylabel('Davies-Bouldin Index')
plt.title('Davies-Bouldin Index For Optimal k')
plt.grid()
plt.savefig("davies_bouldin_all.png", facecolor="white")
plt.show()

In [ ]:
plt.plot(K, calinski_harabasz, 'bx-')
plt.xlabel('k')
plt.ylabel('Calinski-Harabasz Index')
plt.title('Calinski-Harabasz Index For Optimal k')
plt.grid()
plt.savefig("calinski_harabasz_all.png", facecolor="white")
plt.show()

In [ ]:
plt.plot(K, silhouette_coefficient, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette Coefficient')
plt.title('Silhouette Coefficient For Optimal k')
plt.grid()
plt.savefig("silhouette_all.png", facecolor="white")
plt.show()

In [ ]:
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.grid()
plt.savefig("elbow_all.png", facecolor="white")
plt.show()

In [ ]:
from scipy.spatial.distance import cdist
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 30)
  
for k in K:
    # Building and fitting the model
    kmeanModel = BisectingKMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
  
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / X.shape[0])
    inertias.append(kmeanModel.inertia_)
  
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:

plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.grid()
plt.savefig("distorion_all.png", facecolor="white")
plt.show()